## Import libs

In [ ]:
import numpy as np
import torch
import gpytorch
from matplotlib import pyplot as plt 
import os 
import pandas as pd 
import seaborn as sns
import r_pca # Credit to dganguli (https://github.com/dganguli/robust-pca)

## Set up vars

In [ ]:
useGPU = True # Use GPU?
random_test = False # Leave this for now 

# For Model Training
train_new = False # Training new, False if you want to load existing vars
training_iterations = 5000 # Total training iterations
swa_startpt = 3000 # When to start saving weights for Stochastic weight averaging 
interval_checkpt = 50 # Interval for internal MAE tracking 
swa_load_num = 2 # How many weights for SWA

In [ ]:
# Set up GPU usage , 60 GB requiresd
if useGPU == True: # 6 is for personal use 
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    output_device = torch.device('cuda:0')
    n_devices = torch.cuda.device_count()
    output_device = torch.device('cuda:0')
    print('Planning to run on {} GPUs'.format(n_devices))
    
elif useGPU == False: 
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    output_device = torch.device('cpu')
    print('Using CPU')
else: 
    raise ValueError("Incorrect value of useGPU variable")

## Utilities

In [ ]:
def save_checkpoint(model, optimizer, history, iters, fileName):
    torch.save({'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'history': history,
                'iterations': iters
                },
            fileName)

## Import data and pre-process

In [ ]:
# Useful variables 
random_test = False
y_scale = 10

# Read data 
""" Make sure to unpack the rar file"""
data_raw = pd.read_csv('./data/ASPIRE_subsample_data.csv', low_memory=False) # Read csv 

# Pre-process data 
def detect_num(s):
    return any(i.isdigit() for i in s)
## Identify end of coordinates  
for num_idx in range(data_raw.columns.shape[0]):
    out_ = detect_num(data_raw.columns[num_idx])
    if out_ == False:
        num_xc_str = data_raw.columns[num_idx-1]
        num_xc = int(num_xc_str.split('_')[-1])
        break
## Extract column var names for airfoil geometry (z)
zu_str = []
zl_str = []
for i in range(1, num_xc+1):
    zu_str.append('z_u_' + str(i))
    zl_str.append('z_l_' + str(i))
## Define remaining variables to be extracted
rem_str = ['alpha', 'M', 'xc', 'yc'] # angle of attach, Mach, conformal x, conformal y
input_idx = zu_str + zl_str + rem_str

# Convert input data to tensor
input_df = data_raw[input_idx]
X = torch.Tensor(input_df.values)
# X[:,-2] = torch.arcsin(X[:,-2] )
X[:, -4] = torch.deg2rad(X[:, -4]) # Convert AoA to radians
X[:, :-4] *= y_scale
meanX = torch.mean(X, axis=0)
meanX[:] = 0 # unnormalized in this case 
X -= meanX

noise = (torch.Tensor(data_raw['noise'])*y_scale)**2 # Define noise values from experiments

# Extract supplementary data
af = data_raw['af']
cat = data_raw[['symmetry', 'supercritical']]
af_unique = np.unique(af)

# Train - test split per airfoil 
if random_test: 
    # Randomly select test set 
    from sklearn.model_selection import train_test_split
    train_afu, test_afu = train_test_split(af_unique, test_size = .1, random_state = 1) # fix this later
elif random_test == False:
    # Manual override of the test set 
    test_afu = ['SC 1095','Supercritical airfoil 9a', 'NACA 63-415'] # 'RISO-A1-21'
## Remove manually chosen af from af list
train_afu = np.delete(af_unique, np.argwhere(af_unique==test_afu[0]))
for i in range(1, len(test_afu)):
    train_afu = np.delete(train_afu, np.argwhere(train_afu==test_afu[i])) 
## Identify corresponding indices
train_idx = af.isin(train_afu).values
test_idx = af.isin(test_afu).values
## Only use subset?
def get_subset(train_indices, test_indices, subset_bounds):
    M_bounds, A_bounds = subset_bounds[0], subset_bounds[1]
    if subset_bounds[0] is not None:
        train_idx = np.logical_and(input_df['M'] <= M_bounds[1], train_indices)
        train_idx = np.logical_and(input_df['M'] >= M_bounds[0], train_idx)
        test_idx =  np.logical_and(input_df['M'] <= M_bounds[1], test_indices)
        test_idx =  np.logical_and(input_df['M'] >= M_bounds[0], test_idx)
    else:
        train_idx = train_indices
        test_idx = train_indices
    if subset_bounds[1] is not None:
        train_idx = np.logical_and(input_df['alpha'] <= A_bounds[1], train_idx)
        train_idx = np.logical_and(input_df['alpha'] >= A_bounds[0], train_idx)
        test_idx =  np.logical_and(input_df['alpha'] <= A_bounds[1], test_idx)
        test_idx =  np.logical_and(input_df['alpha'] >= A_bounds[0], test_idx)
    return train_idx, test_idx
train_idx, test_idx = get_subset(train_idx, test_idx, [(0.0, 0.73), (-4.0, 12.0)])

# Pre-process training targets 
y = torch.Tensor(data_raw['Cp'].values) * y_scale
y_mean = torch.mean(y)
y -= y_mean

# Train - test split 
train_x, test_x, train_y, test_y, train_af, test_af = X[train_idx], X[test_idx], y[train_idx], y[test_idx], data_raw['af'][train_idx], data_raw['af'][test_idx]
train_noise, test_noise, train_cat, test_cat = noise[train_idx], noise[test_idx], cat[train_idx], cat[test_idx]
## push to output device 
train_x, train_y  = train_x.to(output_device), train_y.to(output_device)

## Define DKL GP Model

In [ ]:
from gpytorch.priors import NormalPrior

data_dim = train_x.size(-1)
nn_dims = [1000, 1000, 500, 50, 10] 

# Define model 
## Main DKL Model
class DKL_GP(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood):
            super(DKL_GP, self).__init__(train_x, train_y, likelihood)
            
            # Mean Module
            self.mean_module = gpytorch.means.ConstantMean()
            
            # Covariance module 
            self.covar_module = gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.keops.MaternKernel(nu=5/2, ard_num_dims=nn_dims[-1]),
                )

            # NN Feature Extractor Module
            self.feature_extractor = LargeFeatureExtractor()
            self.scale_to_bounds = gpytorch.utils.grid.ScaleToBounds(-1.0, 1.0) # scale the feature extractor outputs 

        def forward(self, x):
            # We're first putting our data through a deep net (feature extractor)
            projected_x = self.feature_extractor(x) # .to(output_device)
            projected_x = self.scale_to_bounds(projected_x)  # Make the NN values "nice" .to(output_device)

            mean_x = self.mean_module(projected_x)
            covar_x = self.covar_module(projected_x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
## Feature Extractor 
class LargeFeatureExtractor(torch.nn.Sequential):
    def __init__(self):
        super(LargeFeatureExtractor, self).__init__()
        self.add_module('linear1', torch.nn.Linear(data_dim, nn_dims[0])) # __
        self.add_module('relu1', torch.nn.ReLU())
        self.add_module('dropout1', torch.nn.Dropout(0.2))
        self.add_module('linear2', torch.nn.Linear(nn_dims[0], nn_dims[1]))
        self.add_module('relu2', torch.nn.ReLU())
        self.add_module('dropout2', torch.nn.Dropout(0.2))
        self.add_module('linear3', torch.nn.Linear(nn_dims[1], nn_dims[2]))
        self.add_module('relu3', torch.nn.ReLU())
        self.add_module('dropout3', torch.nn.Dropout(0.2))
        self.add_module('linear4', torch.nn.Linear(nn_dims[2], nn_dims[3]))
        if len(nn_dims) > 4:
            self.add_module('relu4', torch.nn.ReLU())
            self.add_module('dropout4', torch.nn.Dropout(0.2))
            self.add_module('linear5', torch.nn.Linear(nn_dims[3], nn_dims[4]))
        
# Define model & likelihood  
noise_prior = NormalPrior(0.0, 1.0)
likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(noise=train_noise, learn_additional_noise=True, noise_prior=noise_prior) 
likelihood.second_noise = 0.3
model = DKL_GP(train_x, train_y, likelihood)
## Push to output device 
model = model.to(output_device)
model.feature_extractor = model.feature_extractor.to(output_device)
likelihood = likelihood.to(output_device)

# Define Optimizer  
lr = 1e-3
decay = 1e-4 
optimizer = torch.optim.Adam([
    {'params': model.parameters()},
], lr=lr) #  , weight_decay=decay
from torch.optim.lr_scheduler import StepLR, ConstantLR
scheduler = StepLR(optimizer, step_size=1000, gamma = 0.5)#ConstantLR(optimizer, factor=1.0, total_iters=800)

# Set up run or load checkpoint 
load_checkpoint_bool = False # Set this to true if you want to load the best weights

def load_checkpoint(file_path, model, optimizer):
    checkpt = torch.load(file_path, map_location=output_device) 
    model.load_state_dict(checkpt['model_state_dict'])
    optimizer.load_state_dict(checkpt['optimizer_state_dict'])
    history_values = checkpt['history']
    return history_values

def calc_err(true_val, pred_val, err_type = 'MAE'):
        if err_type == 'MAE':
            err = torch.mean(torch.abs(true_val - pred_val))
        if err_type == 'MSE':
            # not implemented yet 
            err = 1
        return err
    
best_loss = float('inf') # loss
best_model_state_dict = None
iters = 0
iters_checkpt = 0
if 'history_values' in locals(): # history
    del history_values

### Run training 

In [ ]:
# Due to the very large covariance matrix, approx 60 GB GPU is required to train the model fully.
import tqdm.notebook as tn 
torch.cuda.empty_cache()

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Set up hyperparameters history
if 'history_values' not in locals():
    history_values = {
        'test_err': np.array([]),
        'train_err': np.array([]),
        'mll': np.array([]),
        'mll_all': np.array([]),
        'n': np.array([])
    }
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model) 
save_tag = 'output'

mem = []
mem_model = []
save_pt_mll = []
save_pt_test = []
save_pt_train = []
save_pt_iter = []

best_loss = float('inf') # loss
best_model_state_dict = None
iters = 0
iters_checkpt = 0

def train_and_save_checkpoints(begin_cycle): 
    global best_loss, best_model_state_dict, best_optim_state_dict, iters, iters_checkpt, mem, scheduler
    sub_iter = 0
    trpz = []
    iterator = tn.tqdm(range(training_iterations))

    for i in iterator:
        model.train()
        likelihood.train()
        
        # Zero backprop gradients
        optimizer.zero_grad()
     
        # Get output from model
        output = model(train_x)
        
        # Calculate loss 
        loss = -mll(output, train_y)  
        loss.backward()
        iterator.set_postfix(loss=loss.item())
        history_values['mll_all'] = np.append(history_values['mll_all'], loss.cpu().detach().numpy())
        
        # Save checkpoint 
        if (i+1)%interval_checkpt == 0:
            model.eval()
            likelihood.eval()
            with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.settings.fast_pred_var():
                # Calculate MAE 
                validation_results = model(test_x.to(output_device)) # temp_model(temp_test)#
                train_results = model(train_x) #  temp_model(temp_train)#
                history_values['mll'] = np.append(history_values['mll'], loss.cpu().detach().numpy())
                history_values['test_err'] = np.append(history_values['test_err'], calc_err(validation_results.mean.cpu(), test_y)) # Force cpu 
                history_values['train_err'] = np.append(history_values['train_err'], calc_err(train_results.mean.cpu(), train_y.cpu())) 
                history_values['n'] = np.append(history_values['n'], iters)
                
                # Print Losses and MAE 
                print('Loss: {}'.format(history_values['mll'][iters_checkpt]) + 
                    ' / Train MAE: {}'.format(history_values['train_err'][iters_checkpt]) + 
                    ' / Test MAE: {}'.format(history_values['test_err'][iters_checkpt]) + 
                    ' / Noise: {}'.format(np.sqrt(likelihood.second_noise_covar.noise.item())))
                mem.append(history_values['test_err'][iters_checkpt])
                mem_model.append([model, optimizer, history_values, iters])
                trpz.append(torch.abs(torch.trapz(validation_results.mean.cpu()) - torch.trapz(test_y)))
                # Continuously update minimum 
                
                min_idx = np.argmin(mem)
                min_idx_sub = len(mem)-1-np.argmin(mem)
                fileDir = './' + gen_save_string(save_tag, lr, 0.0, nn_dims, (history_values['mll'][iters_checkpt-min_idx_sub], history_values['train_err'][iters_checkpt-min_idx_sub], 
                                                                                            history_values['test_err'][iters_checkpt-min_idx_sub]), iters_checkpt-min_idx_sub)
                
                if sub_iter >= 199:
                    save_pt_mll.append(history_values['mll'][iters_checkpt-min_idx_sub])
                    save_pt_train.append(history_values['train_err'][iters_checkpt-min_idx_sub])
                    save_pt_test.append(history_values['test_err'][iters_checkpt-min_idx_sub])
                    save_pt_iter.append(iters_checkpt-min_idx_sub)
                    save_checkpoint(model=mem_model[min_idx][0], optimizer=mem_model[min_idx][1], history=mem_model[min_idx][2], fileName=fileDir, iters=mem_model[min_idx][3])
                    optimizer.param_groups[0]['lr'] = 1e-3
                    scheduler = StepLR(optimizer, step_size=500, gamma=0.999)
                    mem, trpz = [], []
                    sub_iter = 0 
                    print('New cycle')

                iters_checkpt += 1 
        optimizer.step()
        scheduler.step()
        if i >= begin_cycle:
            sub_iter += 1 
        iters += 1 

if train_new:
    # Run Training 
    train_and_save_checkpoints(swa_startpt)

    # Plot Results 
    f, ax = plt.subplots(1,2, figsize=(12, 4))
    # Loss vs iterations
    ax[0].semilogy(history_values['mll_all'])
    ax[0].set_xlabel('Iterations')
    ax[0].set_ylabel('Marginal Log Likelihood')
    ax[0].title.set_text('Loss vs. Iterations')

    # Validation error vs iterations
    ax[1].semilogy(np.arange(interval_checkpt, iters+1, interval_checkpt), np.array(history_values['test_err'][:]),'--', label='Test Data')
    ax[1].legend()

    # Labels
    ax[1].set_xlabel('Iterations')
    ax[1].set_ylabel('MAE')
    ax[1].title.set_text('MAE vs. Iterations')
    plt.show()

## Visualize results

### Analysis sub-library

In [ ]:
def gen_test_data(airfoil, alpha, mach, num_points = 601, manual_point = None, out_x=False):
    global af, data_raw, meanX
    if manual_point is not None:
        num_points = manual_point.shape[0]

    if isinstance(airfoil, str):
        # if string, read airfoil info from training data
        target_af_idx = np.argwhere(af.isin([airfoil]).values)[0]
        target_af_geom = torch.Tensor(data_raw[zu_str+zl_str].values[target_af_idx, :])*y_scale - meanX[:28*2]
        target_af_geom = target_af_geom.tile((num_points,1))
        
    else:
        # manual loading, implement later
        target_xloc_u = torch.Tensor(airfoil[0]).reshape((1,-1)) #np.flip([0, 0.25, 0.75, 1.0, 1.5, 2.0, 2.5, 5.0, 7.5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100])/100
        target_xloc_l = torch.Tensor(airfoil[1]).reshape((1,-1))#np.array([0, 0.25, 0.75, 1.0, 1.5, 2.0, 2.5, 5.0, 7.5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]/100)
        target_af_geom = torch.hstack((target_xloc_u, target_xloc_l))*y_scale - meanX[:28*2]
        target_af_geom = target_af_geom.tile((num_points,1))
    # Final output 
    if manual_point is None:
        # Define others
        xc_u = torch.linspace(-1, 1, num_points//2+1).reshape((num_points//2+1,1)) #torch.linspace(-1.0, 1.0, num_points//2).reshape((num_points//2,1)) # raw x/c
        xc_l = torch.linspace((1.0 - (xc_u[1].item()-xc_u[0].item())), -1, num_points//2).reshape((num_points//2,1))
        # theta = torch.arccos(xc)#torch.arccos(2*torch.abs(xc)-1) # conformal mapping angle
        yc_u = torch.sin(torch.arccos(xc_u))#**2 # y/c
        yc_l = -torch.sin(torch.arccos(xc_l))#**2# y/c
        a = torch.ones((num_points, 1))*alpha# angle of attack
        m = torch.ones((num_points, 1))*mach# mach
    
        test_array = torch.hstack((target_af_geom, a, m, torch.vstack((xc_u, xc_l)), torch.vstack((yc_u, yc_l)))).to(output_device) # torch.vstack((xc, xc))
    else: 
        a = torch.ones((num_points, 1))*alpha # angle of attack
        m = torch.ones((num_points, 1))*mach # mach
        test_array = torch.hstack((target_af_geom, a, m, manual_point[:, 0].reshape((-1,1)), manual_point[:, 1].reshape((-1,1)))).to(output_device)
    
    if out_x == False:
        return test_array 
    elif out_x == True:
        return test_array, (xc_u, xc_l)

def plot_posterior(xc, posterior_predictive, scale = 1, mean = 0, cutoff = None, color='r'):
    f, ax = plt.subplots(1,1)
    
    # obtain cutoff 
    if cutoff is None:
        cutoff = xc.shape[0]//2
    
    xc_u, xc_l = (xc[:cutoff].cpu()+1)/2, (xc[cutoff:].cpu()+1)/2#xc[cutoff:].cpu()
    
    post_mean, post_std = posterior_predictive.mean.cpu(), torch.sqrt(torch.diag(posterior_predictive.covariance_matrix.cpu()))
    mu_u, mu_l = (post_mean[:cutoff] + mean)/scale, (post_mean[cutoff:] + mean)/scale
    std_u, std_l = post_std[:cutoff]/scale, post_std[cutoff:]/scale
    
    # Plot posterior mean
    ax.plot(xc_u, mu_u, color=color, linestyle='-', label = 'DKL GP model', linewidth=2.0)
    ax.plot(xc_l, mu_l, color=color, linestyle='--')
    ax.fill_between(xc_u, mu_u - 2*std_u, mu_u + 2*std_u, color='lightgray', label = 'Predicted $2\sigma$', linewidth=2.0)
    ax.fill_between(xc_l, mu_l - 2*std_l, mu_l + 2*std_l, color='lightgray', linewidth=2.0)
    # ax.set_ylim([-3.0, 1.5])
    return (f, ax), (xc_u, xc_l), (mu_u, mu_l), (std_u, std_l)

def aggregate_posterior(test_data, weights_list, get_cl=False):
    n_ = len(weights_list)
    mean = torch.zeros(test_data.shape[0])
    covar = torch.zeros(test_data.shape[0], test_data.shape[0]) 
    for n in range(0, n_):
        load_checkpoint('./weights/' + weights_list[n], model, optimizer)
        with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.settings.fast_pred_var():
            # print(test_data.get_device())
            preds = model(test_data)
            mean += preds.mean.cpu()
            covar += preds.covariance_matrix.cpu()
            
            if get_cl:
                1 
    
    mean = mean/n_
    covar = covar/n_
    agg_posterior_predictive_dist = gpytorch.distributions.MultivariateNormal(mean, covar + torch.eye(mean.shape[0])*1e-4)
    
    if get_cl:
        return 1
    elif get_cl == False:
        return agg_posterior_predictive_dist 

def agg_covar(train_data, test_data, weights_list, get_kxx = True):
    n_ = len(weights_list)
    # mean = torch.zeros(test_data.shape[0])
    Kxx = torch.zeros(train_data.shape[0], train_data.shape[0])
    Kxs = torch.zeros(train_data.shape[0], test_data.shape[0])
    Kss = torch.zeros(test_data.shape[0], test_data.shape[0])
    for n in range(0, n_):
        load_checkpoint('./weights/' + weights_list[n], model, optimizer)
        projected_xtrain = model.feature_extractor(train_data)
        projected_xtrain = model.scale_to_bounds(projected_xtrain)
        projected_xtest = model.feature_extractor(test_data)
        projected_xtest = model.scale_to_bounds(projected_xtest)
        
        with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.settings.fast_pred_var():
            jitter = 1e-4 * torch.eye(train_x.shape[0])
            if get_kxx:
                Kxx += (model.covar_module(projected_xtrain).evaluate().cpu() + torch.diag(likelihood.noise.cpu()) + jitter).detach().numpy()
            Kxs += (model.covar_module(projected_xtrain, projected_xtest).evaluate().cpu()).detach().numpy()
            Kss += (model.covar_module(projected_xtest, projected_xtest).evaluate().cpu()).detach().numpy()
    Kxx = Kxx/n_
    Kxs = Kxs/n_
    Kss = Kss/n_
    if get_kxx:
        return Kxx, Kxs, Kss
    else: 
        return Kxs, Kss

def aggregate_latent_vars(test_data, weights_list):
    n_ = len(weights_list)
    mean = torch.zeros((test_data.shape[0], nn_dims[-1]))
    for n in range(0, n_):
        load_checkpoint('./weights/' + weights_list[n], model, optimizer)
        with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.settings.fast_pred_var():
            preds = model.feature_extractor(test_data)
            preds = model.scale_to_bounds(preds)
            mean += preds.cpu()
    
    mean = mean/n_
    return mean 

def validate_predictions(x_data, y_data, af_df, test_case, num_cases, weights_list = None, save_figs = False):
    input_var = []
    output_var = []
    
    af_u = af_df.unique()
    subtest_af = [af_u[test_case]]
    subtest_x = x_data[af_df.isin(subtest_af).values]
    subtest_y = y_data[af_df.isin(subtest_af).values]

    # Evaluate
    if weights_list is None:
        weights_list = []
        for n in range(0, 5):
            weights_list.append(gen_save_string(save_tag, lr, decay, nn_dims, (history_values['mll'][iters_checkpt-1-n], 
                                                                    history_values['train_err'][iters_checkpt-1-n], 
                                                                    history_values['test_err'][iters_checkpt-1-n]), 
                                    iters_checkpt-1-n))
    preds_subtest = aggregate_posterior(subtest_x.to(output_device), weights_list)
    print('Evaulating posterior predictive distribution for airfoil ' + subtest_af[0] + '...')
    print('Subtest MAE: {}'.format(calc_err(preds_subtest.mean.cpu(), subtest_y.cpu())))

    targetAF = af_u[test_case]
        
    temp_ind_af = np.where(af_df.values == targetAF)[0]
    temp_arr = x_data[temp_ind_af]

    unique_AM_pair = np.unique(temp_arr[:,-4:-2], axis=0)
    if num_cases > unique_AM_pair.shape[0]:
        num_cases = unique_AM_pair.shape[0]
        
    for j in np.arange(0, num_cases):
        plt.figure()

        # Generate predictor and make predictions
        ## Predictor
        test_airfoil = gen_test_data(af_u[test_case], unique_AM_pair[j][0], unique_AM_pair[j][1])
        input_var.append(test_airfoil)
        temp_ind = temp_ind_af[torch.logical_and(temp_arr[:, -4] == unique_AM_pair[j,0], temp_arr[:, -3] == unique_AM_pair[j,1])]
        
        ## Evaulate Posterior Predictive 
        sample_airfoil_pred = aggregate_posterior(test_airfoil.to(output_device), weights_list)
        output_var.append(sample_airfoil_pred)

        # Plot result
        ## posterior
        (f, ax), (xc_u, xc_l), (mu_u, mu_l), (std_u, std_l) = plot_posterior(test_airfoil[:,-2], sample_airfoil_pred, scale = y_scale, mean = y_mean)
        ## experimental validation
        temp_plot_x = ((x_data[temp_ind, -2]+1)/2).cpu().detach().numpy()
        temp_plot_y = (y_data[temp_ind].cpu()+y_mean)/y_scale
        temp_plot_noise = noise[temp_ind]
        ax.errorbar(temp_plot_x, temp_plot_y, yerr=2*np.sqrt(temp_plot_noise)/y_scale, fmt='k.', capsize=2, label='Expt., Flemming 1984') #  $C_p \pm 2\sigma$
        ax.invert_yaxis()
        plt.xlabel('x/c')
        plt.ylabel('$C_p$')
        # plt.legend()
        plt.title(targetAF +'\n' + r'$\alpha$ = ' + str(np.round(np.rad2deg(unique_AM_pair[j,0]),2)) + r'$^\circ,$' + r' $M_\infty = $' + str(unique_AM_pair[j,1]))
        if save_figs == True:
            plt.savefig(targetAF+'_'+'a'+str(np.round(np.rad2deg(unique_AM_pair[j,0]),2))+'_'+str(unique_AM_pair[j,1])+'.png', bbox_inches='tight')
        plt.show() 
    return input_var, output_var

### Visualize Predictions on Test set

In [ ]:
model.eval()
likelihood.eval()
weights_ = []

# If train new 
if train_new:
    weight_sweep_inds = np.lexsort((save_pt_train, save_pt_test, save_pt_mll))
    for i in range(0, swa_load_num):
        load_str = gen_save_string(save_tag, lr, 0.0, nn_dims, (save_pt_mll[weight_sweep_inds[i]], 
                                                                        save_pt_train[weight_sweep_inds[i]], 
                                                                        save_pt_test[weight_sweep_inds[i]]), 
                                    save_pt_iter[weight_sweep_inds[i]], override_date=None)
        weights_.append(load_str)
else:
    weights_ = ['weights_1', 
                'weights_2',
                'weights_3',] 

num_figs = 1
# 0: NACA 63-415, 1: Supercritical airfoil 9a, 2: SC1095
_, _ = validate_predictions(test_x, test_y, test_af, 2, num_figs, weights_list=weights_, save_figs=False)

## Data visualization


Distribution of data wrt airfoil family

In [ ]:
data = data_raw
manual_airfoil_series = ['NACA 4 series', 'NACA 64A Series', 'NASA Supercritical airfoil','Bell family', 'RISO-A family']
        # Category vs. Number at varying alpha 
A_range = np.arange(-16.0, 31.0, 1.0)
M_range = np.arange(0.0+0.05, 1.0, 0.1)
result = np.zeros((M_range.shape[0], A_range.shape[0]))
result_af_family = np.zeros((M_range.shape[0], A_range.shape[0], len(manual_airfoil_series)))
results2 = []
counter = 0

for i in range(0, M_range.shape[0]):
    new_ind_M = np.abs(data['M']-M_range[i]) < 0.05
    for j in range(0, A_range.shape[0]):
        new_ind_A = (data['alpha']-A_range[j] < 0.5) & (data['alpha']-A_range[j] >= -0.5)
        new_ind = np.logical_and(new_ind_M, new_ind_A)
        result[i, j] = len(data['af'][new_ind].unique())
        
        # Catergorize for stacked bar plot
        if len(data['family'][new_ind]) != 0 :
            for ii in range(0, len(manual_airfoil_series)):
                if (data['family'][new_ind]).isin([manual_airfoil_series[ii]]).any():
                    temp_idx = (data['family'][new_ind]).isin([manual_airfoil_series[ii]])
                    result_af_family[i, j, ii] = len(data['af'][new_ind][temp_idx].unique()) 
        
        results2.append([M_range[i], A_range[j], result[i, j]])
results2 = np.vstack(results2)
# Make figure
f = plt.figure(figsize=(24,6))
gs = f.add_gridspec(2, 3, width_ratios=(1, 8, 0.65), height_ratios=(1, 2.5),
                    left=0.05, right=0.8, bottom=0.1, top=0.9,
                    wspace=0.05, hspace=0.05)
ax = f.add_subplot(gs[1, 1])
ax_histx = f.add_subplot(gs[0, 1], sharex=ax)
ax_histy = f.add_subplot(gs[1, 2], sharey=ax)

# Plot main
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib
# prop_cycle = plt.rcParams['axes.prop_cycle']
colors_plt = ['#0072BD', '#D95319', '#EDB120', '#7E2F8E', '#77AC30', '#4DBEEE', '#A2142F']

cmap = plt.get_cmap('Reds')
colors = cmap(np.linspace(0, 0.7, 256))
colors[0, :] = [1,1,1,1]
custom_cmap = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors)

im = ax.imshow(result, cmap=custom_cmap, aspect='auto', origin='lower')
ax.set_yticks(np.arange(0, M_range.shape[0])-0.5, labels=np.round(M_range-0.05, 2))
ax.set_xticks(np.arange(0, A_range.shape[0],2), labels=np.round(A_range[::2], 2))

for i in range(0, M_range.shape[0]):
    for j in range(0, A_range.shape[0]):
        text = ax.text(j, i, int(result[i, j]) if not np.isnan(result[i,j]) and result[i,j] != 0 else '',
                        ha="center", va="center_baseline", color="k")

ax.set_xlabel(r'Angle of attack, $\alpha$ [deg]', fontsize=16)
ax.set_ylabel('Mach number, M', fontsize=16)

ax_histx.tick_params(axis="x", labelbottom=False)
ax_histy.tick_params(axis="y", labelleft=False)

ax_histx.bar(np.arange(0, A_range.shape[0]), np.sum(result_af_family[:,:,0],axis=0),
             label = 'NACA 4-Digit', color=colors_plt[0])
ax_histx.bar(np.arange(0, A_range.shape[0]), np.sum(result_af_family[:,:,1],axis=0), bottom=np.sum(result_af_family[:,:,0],axis=0),
             label = 'NACA 64A', color=colors_plt[1])
ax_histx.bar(np.arange(0, A_range.shape[0]), np.sum(result_af_family[:,:,2],axis=0), bottom=np.sum(result_af_family[:,:,0],axis=0)
             +np.sum(result_af_family[:,:,1],axis=0), label='NASA Supercritical airfoil', color=colors_plt[2])
ax_histx.bar(np.arange(0, A_range.shape[0]), np.sum(result_af_family[:,:,3],axis=0), bottom=np.sum(result_af_family[:,:,0],axis=0)+np.sum(result_af_family[:,:,1],axis=0)
             +np.sum(result_af_family[:,:,2],axis=0), label='DU Series', color=colors_plt[3])
ax_histx.bar(np.arange(0, A_range.shape[0]), np.sum(result_af_family[:,:,4],axis=0), bottom=np.sum(result_af_family[:,:,0],axis=0)+np.sum(result_af_family[:,:,1],axis=0)
             +np.sum(result_af_family[:,:,2],axis=0)+np.sum(result_af_family[:,:,3],axis=0), label='RISO-A Family', color=colors_plt[4])
ax_histx.bar(np.arange(0, A_range.shape[0]), np.sum(result,axis=0),zorder=0, label='Others', color=colors_plt[5])
ax_histy.barh(np.arange(0, M_range.shape[0]), np.sum(result_af_family[:,:,0],axis=1),
             label = 'NACA 4-Digit Series', color=colors_plt[0])
ax_histy.barh(np.arange(0, M_range.shape[0]), np.sum(result_af_family[:,:,1],axis=1), left=np.sum(result_af_family[:,:,0],axis=1),
             label = 'NACA 64A Series', color=colors_plt[1])
ax_histy.barh(np.arange(0, M_range.shape[0]), np.sum(result_af_family[:,:,2],axis=1), left=np.sum(result_af_family[:,:,0],axis=1)
             +np.sum(result_af_family[:,:,1],axis=1), label='NASA Supercritical', color=colors_plt[2])
ax_histy.barh(np.arange(0, M_range.shape[0]), np.sum(result_af_family[:,:,3],axis=1), left=np.sum(result_af_family[:,:,0],axis=1)+np.sum(result_af_family[:,:,1],axis=1)
             +np.sum(result_af_family[:,:,2],axis=1), label='Bell', color=colors_plt[3])
ax_histy.barh(np.arange(0, M_range.shape[0]), np.sum(result_af_family[:,:,4],axis=1), left=np.sum(result_af_family[:,:,0],axis=1)+np.sum(result_af_family[:,:,1],axis=1)
             +np.sum(result_af_family[:,:,2],axis=1)+np.sum(result_af_family[:,:,3],axis=1), label='RISO-A', color=colors_plt[4])
ax_histy.barh(np.arange(0, M_range.shape[0]), np.sum(result,axis=1),zorder=0, label='Others', color=colors_plt[5])
ax_histy.legend(bbox_to_anchor=(-0.35, -0.45), ncol=6)

w=1.0
h=0.03

w1=w*(ax.get_position().x1-ax.get_position().x0)
x1=0.135
y1=ax.get_position().y0-5*h

cax = f.add_axes([x1,y1,w1,h])
cbar = plt.colorbar(im, cax=cax, orientation='horizontal', ticks = np.arange(0, 15.1), boundaries=np.arange(0, 15.1))
cbar.set_label('Number of airfoils') 
# plt.subplots_adjust(left=0.15, right=0.95, bottom=0.2, top=0.95)
# plt.savefig("./Saved Figures/2_avail_data_family.pdf", dpi=300, bbox_inches='tight') # 
plt.show()

Distribution of data wrt airfoil usage

In [ ]:
data = data_raw
        # Category vs. Number at varying alpha 
A_range = np.arange(-16.0, 31.0, 1.0)
M_range = np.arange(0.0+0.05, 1.0, 0.1)
manual_airfoil_type = ['General', 'Rotor', 'Wind Turbine']
result = np.zeros((M_range.shape[0], A_range.shape[0]))
result_af_type = np.zeros((M_range.shape[0], A_range.shape[0], len(manual_airfoil_type)))
results2 = []
counter = 0

for i in range(0, M_range.shape[0]):
    new_ind_M = np.abs(data['M']-M_range[i]) < 0.05
    for j in range(0, A_range.shape[0]):
        new_ind_A = (data['alpha']-A_range[j] < 0.5) & (data['alpha']-A_range[j] >= -0.5)
        new_ind = np.logical_and(new_ind_M, new_ind_A)
        result[i, j] = len(data['af'][new_ind].unique())
        
        # Catergorize for stacked bar plot
        if len(data['usage'][new_ind]) != 0 :
            for ii in range(0, len(manual_airfoil_type)):
                if (data['usage'][new_ind]).isin([manual_airfoil_type[ii]]).any():
                    temp_idx = (data['usage'][new_ind]).isin([manual_airfoil_type[ii]])
                    result_af_type[i, j, ii] = len(data['af'][new_ind][temp_idx].unique()) 
        
        results2.append([M_range[i], A_range[j], result[i, j]])
results2 = np.vstack(results2)
# Make figure
f = plt.figure(figsize=(24,6))
gs = f.add_gridspec(2, 3, width_ratios=(1, 8, 0.65), height_ratios=(1, 2.5),
                    left=0.05, right=0.8, bottom=0.1, top=0.9,
                    wspace=0.05, hspace=0.05)
ax = f.add_subplot(gs[1, 1])
ax_histx = f.add_subplot(gs[0, 1], sharex=ax)
ax_histy = f.add_subplot(gs[1, 2], sharey=ax)

# Plot main
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib
# prop_cycle = plt.rcParams['axes.prop_cycle']
# colors_plt = prop_cycle.by_key()['color']

cmap = plt.get_cmap('Reds')
colors = cmap(np.linspace(0, 0.7, 256))
colors[0, :] = [1, 1, 1, 1,]
custom_cmap = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors)
im = ax.imshow(result, cmap=custom_cmap, aspect='auto', origin='lower')
ax.set_yticks(np.arange(0, M_range.shape[0])-0.5, labels=np.round(M_range-0.05, 2))
ax.set_xticks(np.arange(0, A_range.shape[0],2), labels=np.round(A_range[::2], 2))

for i in range(0, M_range.shape[0]):
    for j in range(0, A_range.shape[0]):
        text = ax.text(j, i, int(result[i, j]) if not np.isnan(result[i,j]) and result[i,j] != 0 else '',
                        ha="center", va="center_baseline", color="k")

ax.set_xlabel(r'Angle of attack, $\alpha$ [deg]', fontsize=16)
ax.set_ylabel('Mach number, M', fontsize=16)

ax_histx.tick_params(axis="x", labelbottom=False)
ax_histy.tick_params(axis="y", labelleft=False)

ax_histx.bar(np.arange(0, A_range.shape[0]), np.sum(result_af_type[:,:,0],axis=0),
             label = 'General', color=colors_plt[0])
ax_histx.bar(np.arange(0, A_range.shape[0]), np.sum(result_af_type[:,:,1],axis=0), bottom=np.sum(result_af_type[:,:,0],axis=0),
             label = 'Rotor', color=colors_plt[1])
ax_histx.bar(np.arange(0, A_range.shape[0]), np.sum(result_af_type[:,:,2],axis=0), bottom=np.sum(result_af_type[:,:,0],axis=0)
             +np.sum(result_af_type[:,:,1],axis=0), label='Wind Turbine', color=colors_plt[2])
# ax_histx.bar(np.arange(0, A_range.shape[0]), np.sum(result, axis=0), zorder=0, width=1.0)

# ax_histx.legend()
# ax_histy.barh(np.arange(0, M_range.shape[0]), np.sum(result,axis=1))
ax_histy.barh(np.arange(0, M_range.shape[0]), np.sum(result_af_type[:,:,0],axis=1),
             label = 'General', color=colors_plt[0])
ax_histy.barh(np.arange(0, M_range.shape[0]), np.sum(result_af_type[:,:,1],axis=1), left=np.sum(result_af_type[:,:,0],axis=1),
             label = 'Rotor', color=colors_plt[1])
ax_histy.barh(np.arange(0, M_range.shape[0]), np.sum(result_af_type[:,:,2],axis=1), left=np.sum(result_af_type[:,:,0],axis=1)
             +np.sum(result_af_type[:,:,1],axis=1), label='Wind Turbine', color=colors_plt[2])

# ax_histy.barh(np.arange(0, M_range.shape[0]), np.sum(result,axis=1), zorder=0, height=1.0)
ax_histy.legend(bbox_to_anchor=(-4.0, -0.45), ncol=6)

w=1.0
h=0.03

w1=w*(ax.get_position().x1-ax.get_position().x0)
x1=0.137#ax.get_position().x0+w1/2
y1=ax.get_position().y0-5*h
cax = f.add_axes([x1,y1,w1,h])
cbar = plt.colorbar(im, cax=cax, orientation='horizontal', ticks = np.arange(0, 15.1), boundaries=np.arange(0, 15.1))
cbar.set_label('Number of airfoils') 

# plt.savefig("./Saved Figures/2_avail_data_usage.pdf", dpi=300, bbox_inches='tight')
plt.show()

Distribution of data wrt airfoil supercriticality

In [ ]:
data = data_raw
        # Category vs. Number at varying alpha 
A_range = np.arange(-16.0, 31.0, 1.0)
M_range = np.arange(0.0+0.05, 1.0, 0.1)
manual_airfoil_sc = ['-', 'Supercritical']
result = np.zeros((M_range.shape[0], A_range.shape[0]))
result_af_sc = np.zeros((M_range.shape[0], A_range.shape[0], len(manual_airfoil_sc)))
results2 = []
counter = 0

for i in range(0, M_range.shape[0]):
    new_ind_M = np.abs(data['M']-M_range[i]) < 0.05
    for j in range(0, A_range.shape[0]):
        new_ind_A = (data['alpha']-A_range[j] < 0.5) & (data['alpha']-A_range[j] >= -0.5)
        new_ind = np.logical_and(new_ind_M, new_ind_A)
        result[i, j] = len(data['af'][new_ind].unique())
        
        # Catergorize for stacked bar plot
        if len(data['supercritical'][new_ind]) != 0 :
            for ii in range(0, len(manual_airfoil_sc)):
                if (data['supercritical'][new_ind]).isin([manual_airfoil_sc[ii]]).any():
                    temp_idx = (data['supercritical'][new_ind]).isin([manual_airfoil_sc[ii]])
                    result_af_sc[i, j, ii] = len(data['af'][new_ind][temp_idx].unique()) 
        
        results2.append([M_range[i], A_range[j], result[i, j]])
results2 = np.vstack(results2)
# Make figure
f = plt.figure(figsize=(24,6))
gs = f.add_gridspec(2, 3, width_ratios=(1, 8, 0.65), height_ratios=(1, 2.5),
                    left=0.05, right=0.8, bottom=0.1, top=0.9,
                    wspace=0.05, hspace=0.05)
ax = f.add_subplot(gs[1, 1])
ax_histx = f.add_subplot(gs[0, 1], sharex=ax)
ax_histy = f.add_subplot(gs[1, 2], sharey=ax)

# Plot main
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib

cmap = plt.get_cmap('Reds')
colors = cmap(np.linspace(0, 0.7, 256))
colors[0, :] = [1, 1, 1, 1]
custom_cmap = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors)
im = ax.imshow(result, cmap=custom_cmap, aspect='auto', origin='lower')
ax.set_yticks(np.arange(0, M_range.shape[0])-0.5, labels=np.round(M_range-0.05, 2))
ax.set_xticks(np.arange(0, A_range.shape[0],2), labels=np.round(A_range[::2], 2))

for i in range(0, M_range.shape[0]):
    for j in range(0, A_range.shape[0]):
        text = ax.text(j, i, int(result[i, j]) if not np.isnan(result[i,j]) and result[i,j] != 0 else '',
                        ha="center", va="center_baseline", color="k")

ax.set_xlabel(r'Angle of attack, $\alpha$ [deg]', fontsize=16)
ax.set_ylabel('Mach number, M', fontsize=16)

ax_histx.tick_params(axis="x", labelbottom=False)
ax_histy.tick_params(axis="y", labelleft=False)

ax_histx.bar(np.arange(0, A_range.shape[0]), np.sum(result_af_sc[:,:,0],axis=0),
             label = 'General', color=colors_plt[0])
ax_histx.bar(np.arange(0, A_range.shape[0]), np.sum(result_af_sc[:,:,1],axis=0), bottom=np.sum(result_af_sc[:,:,0],axis=0),
             label = 'Supercritical', color=colors_plt[1]) 

# ax_histx.legend()
# ax_histy.barh(np.arange(0, M_range.shape[0]), np.sum(result,axis=1))
ax_histy.barh(np.arange(0, M_range.shape[0]), np.sum(result_af_sc[:,:,0],axis=1),
             label = 'General', color=colors_plt[0])
ax_histy.barh(np.arange(0, M_range.shape[0]), np.sum(result_af_sc[:,:,1],axis=1), left=np.sum(result_af_sc[:,:,0],axis=1),
             label = 'Supercritical', color=colors_plt[1])

ax_histy.barh(np.arange(0, M_range.shape[0]), np.sum(result,axis=1),zorder=0)
ax_histy.legend(bbox_to_anchor=(-4.7, -0.45), ncol=6)

w=1.0
h=0.03

w1=w*(ax.get_position().x1-ax.get_position().x0)
x1=0.1355#ax.get_position().x0+w1/2
y1=ax.get_position().y0-5*h

cax = f.add_axes([x1,y1,w1,h])
cbar = plt.colorbar(im, cax=cax, orientation='horizontal', ticks = np.arange(0, 15.1), boundaries=np.arange(0, 15.1))
cbar.set_label('Number of airfoils') 
# plt.subplots_adjust(left=0.15, right=0.95, bottom=0.2, top=0.95)
# plt.savefig("./Saved Figures/2_avail_data_supercrit.pdf", dpi=300, bbox_inches='tight')
plt.show()

## Analysis of results

### Robust PCA Analysis of Model Latent Space

Get Latent Variables from all unique airfoils (takes some time to do)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from matplotlib import cm
import matplotlib.lines as mlines

a_sweep_range = np.arange(-4.0, 12.1, 2.0)
m_sweep_range = np.array([0.0, 0.3, 0.6, 0.7])
x_sweep_range = np.linspace(-1.0, 1.0, 601)

af_str_ = af_unique
af_arr = []
af_type = []
latent_vars = []
latent_vars_int = []

input_vars = []
output_cps = []
output_cn = []
conformal = lambda x: np.sin(np.arccos(x))

counter = 0
for af_str in af_str_:
    for m in m_sweep_range:
        for a in a_sweep_range: 
            test_airfoil = gen_test_data(af_str, np.deg2rad(a), m) #
            
            xcu = (test_airfoil[0:301, -2].cpu().detach().numpy().flatten()+1)/2
            xcl = (test_airfoil[300:601, -2].cpu().detach().numpy().flatten()+1)/2

            input_vars.append([a, m])
            latent_vars_ = aggregate_latent_vars(test_airfoil.cuda(), weights_).cpu().detach().numpy()
            latent_vars.append(latent_vars_)
            output_cps_ = aggregate_posterior(test_airfoil.cuda(), weights_).mean.cpu().detach().numpy()
            output_cps.append(output_cps_)
            af_arr.append(af_str)

            latent_vars_int_ = -np.trapz(y=latent_vars_[:301, :], x=xcu, axis=0) + np.trapz(y=np.flip(latent_vars_[300:, :], axis=0), x=np.flip(xcl), axis=0)
            latent_vars_int.append(latent_vars_int_)

            output_cn_ = -np.trapz(y=output_cps_[:301]*y_scale, x=xcu, axis=0) + np.trapz(y=np.flip(output_cps_[300:]*y_scale, axis=0), x=np.flip(xcl), axis=0)
            output_cn.append(output_cn_)
            counter += 1

input_vars = np.vstack(input_vars)          
latent_vars = np.vstack(latent_vars)
af_arr = np.vstack(af_arr)
output_cps = np.vstack(output_cps)
output_cn = np.vstack(output_cn)
latent_vars_int = np.vstack(latent_vars_int)

import pickle 
with open('latent_vars.pkl', 'wb') as file:
    pickle.dump({'input_vars':input_vars,
                 'latent_vars':latent_vars,
                 'af_arr': af_arr,
                 'output_cps': output_cps,
                 'ouput_cn': output_cn,
                 'latent_vars_int':latent_vars_int}, file)

In [ ]:
# Adjusted colormap
cmap_a = plt.get_cmap('seismic')
cmap_m = plt.get_cmap('viridis')
norm_a = (a_sweep_range + a_sweep_range[-1])/(2*a_sweep_range[-1])
norm_m = (m_sweep_range-m_sweep_range[0])/(m_sweep_range[-1]-m_sweep_range[0])
colors_a = cmap_a(norm_a)

colors_m = cmap_m(norm_m)
colors_a = np.tile(colors_a, (af_unique.shape[0]*len(m_sweep_range), 1)) # random_af_sample.shape[0]
colors_m = np.tile(np.repeat(colors_m,len(a_sweep_range), axis=0), (af_unique.shape[0],1))

#### vs. Angle of attack

In [ ]:
a_idx = input_vars[:,1]==0.7 # Set M = 0.70
m_idx = (input_vars[:,0]==4.0).flatten() & (input_vars[:,1]>0.0).flatten()

# Select some airfoils and flow conditions
af1_idx = ((af_arr=='NACA 0012').flatten() | (af_arr=='NACA 0011').flatten()) & a_idx & (input_vars[:,0]<10).flatten()
af2_idx = ((af_arr=='Supercritical Airfoil 26a').flatten() | (af_arr=='Supercritical Airfoil 12').flatten()) & a_idx & (input_vars[:,0]<10).flatten()
af3_idx = ((af_arr=='SSC-A07').flatten() | (af_arr=='SSC-A09').flatten()) & a_idx & (input_vars[:,0]<10).flatten()
pca = PCA(n_components=9)

rpca = r_pca.R_pca(latent_vars_int - np.mean(latent_vars_int)) # - np.mean(latent_vars, axis=0) # , lmbda=0.0
L, S = rpca.fit(max_iter=10000, iter_print=10000, tol=2.0e-2) #  
rX_pca = pca.fit_transform(L) #  # 6.5e-4
print(pca.explained_variance_ratio_.cumsum())

Plot Cumulative Explained Variance Ratio

In [ ]:
f = plt.figure(figsize=(10,6))
ax = f.add_axes([0.16, 0.17, 0.68, 0.8])
ax.bar(np.arange(1, 10), pca.explained_variance_ratio_, color='b', label = 'Individual Exp. Var.')
ax.plot(np.arange(1, 10), pca.explained_variance_ratio_.cumsum(), 'ro-', label = 'Cumulative Exp. Var.',linewidth=2)
ax.set_xlabel('Principal Components', fontsize=32)
ax.set_ylabel('Explained Variance Ratio', fontsize=32)
plt.xticks(fontsize=32)
plt.yticks(fontsize=32)
plt.xticks(np.arange(1,10))
plt.legend(loc='right', fontsize=24)

In [ ]:
from matplotlib import gridspec
f = plt.figure(figsize=(16,10))
spec = gridspec.GridSpec(ncols=4, nrows=3,
                         width_ratios=[4, .01, 2, .1], height_ratios=[1,1,1])
ax1 = f.add_subplot(spec[:, 0], projection='3d')
ax2 = f.add_subplot(spec[0, 2])
ax3 = f.add_subplot(spec[1, 2])
ax4 = f.add_subplot(spec[2, 2])
cbar_ax = f.add_subplot(spec[:, 3])
cbar_ax.axis('off')
sort_by_a = np.argsort(input_vars[a_idx][:, 0])
colors = colors_a
leg_handles = []

    # Plot result
sct = ax1.scatter(rX_pca[a_idx][sort_by_a, 0], rX_pca[a_idx][sort_by_a, 1], rX_pca[a_idx][sort_by_a, 2], c=colors[a_idx][sort_by_a], marker='o', edgecolors='k', s=80, alpha = 1.0, depthshade=True)

ax1.view_init(35, -60)
ax1.yaxis.labelpad = 30
ax1.zaxis.labelpad = 25
ax1.xaxis.labelpad = 20
ax1.set_xlabel('PC1', fontsize=32)
ax1.set_ylabel('PC2', fontsize=32)
ax1.set_zlabel('PC3', fontsize=32)
ax1.tick_params(axis='both', which='major', labelsize=24)

sort_by_ = np.argsort(rX_pca[a_idx][:, 2])
sct = ax2.scatter(rX_pca[a_idx][sort_by_, 0], rX_pca[a_idx][sort_by_, 1], c=colors[a_idx][sort_by_], marker='o', edgecolors='k', s=80)
ax2.set_xlabel('PC1', fontsize=24)
ax2.set_ylabel('PC2', fontsize=24)
ax2.tick_params(axis='both', which='major', labelsize=20)
ax2.set_xlim([-0.4, 0.55])
ax2.set_ylim([-0.3, 0.3])

sort_by_ = np.argsort(rX_pca[a_idx][:, 0])
sct = ax3.scatter(rX_pca[a_idx][sort_by_, 1], rX_pca[a_idx][sort_by_, 2], c=colors[a_idx][sort_by_], marker='o', edgecolors='k', s=80)
ax3.set_xlabel('PC2', fontsize=24)
ax3.set_ylabel('PC3', fontsize=24)
ax3.set_xlim([-0.3, 0.3])
ax3.set_ylim([-0.2, 0.2])
ax3.tick_params(axis='both', which='major', labelsize=20)
sct.set_cmap('seismic') 

cbar = plt.colorbar(sct, ax=cbar_ax, ticks=(a_sweep_range+ a_sweep_range[-1])/(2*a_sweep_range[-1]), location='right', fraction=2.5, boundaries=(np.arange(-4.0, 12.1, 0.1)+ a_sweep_range[-1])/(2*a_sweep_range[-1]))
cbar.set_label(r'Angle of Attack, $\alpha$ [deg]', fontsize=24)
cbar.set_ticklabels(a_sweep_range)
cbar.ax.tick_params(axis='both', which='major', labelsize=20)

sort_by_ = np.argsort(rX_pca[a_idx][:, 1])
sct = ax4.scatter(rX_pca[a_idx][sort_by_, 0], rX_pca[a_idx][sort_by_, 2], c=colors[a_idx][sort_by_], marker='o', edgecolors='k', s=80) 
ax4.set_xlabel('PC1', fontsize=24)
ax4.set_ylabel('PC3', fontsize=24)
ax4.tick_params(axis='both', which='major', labelsize=20)
ax4.set_xlim([-0.4, 0.55])
ax4.set_ylim([-0.2, 0.2])
plt.tight_layout()
# plt.savefig('./Saved Figures/4_latvar_a2.png', dpi=300)

#### vs. Mach number


In [ ]:
from matplotlib import gridspec
f = plt.figure(figsize=(16,10))
spec = gridspec.GridSpec(ncols=4, nrows=3,
                         width_ratios=[4, .1, 2, .1], height_ratios=[1,1,1])
ax1 = f.add_subplot(spec[:, 0], projection='3d')
ax2 = f.add_subplot(spec[0, 2])
ax3 = f.add_subplot(spec[1, 2])
ax4 = f.add_subplot(spec[2, 2])
cbar_ax = f.add_subplot(spec[:, 3])
cbar_ax.axis('off')


sort_by_m = np.flip(np.argsort(input_vars[m_idx][:, 1]))

colors = colors_m
leg_handles = []
    # Plot result
ax1.view_init(35, -60)
sct = ax1.scatter(rX_pca[m_idx][sort_by_m, 0], rX_pca[m_idx][sort_by_m, 1], rX_pca[m_idx][sort_by_m, 2], c=colors[m_idx][sort_by_m], marker='o', edgecolors='k', s=80, depthshade=False)
ax1.yaxis.labelpad = 30
ax1.zaxis.labelpad = 25
ax1.xaxis.labelpad = 20
ax1.set_xlabel('PC1', fontsize=32)
ax1.set_ylabel('PC2', fontsize=32)
ax1.set_zlabel('PC3', fontsize=32)
ax1.tick_params(axis='both', which='major', labelsize=24)

# 

sort_by_ = np.argsort(rX_pca[m_idx][:, 2])
sct = ax2.scatter(rX_pca[m_idx][sort_by_, 0], rX_pca[m_idx][sort_by_, 1], c=colors[m_idx][sort_by_], marker='o', edgecolors='k', s=80)
ax2.set_xlabel('PC1', fontsize=24)
ax2.set_ylabel('PC2', fontsize=24)
ax2.tick_params(axis='both', which='major', labelsize=20)
ax2.set_xticks(np.arange(-0.25, 0.25+.1, 0.25))

sort_by_ = np.argsort(rX_pca[m_idx][:, 0])
sct = ax3.scatter(rX_pca[m_idx][sort_by_, 1], rX_pca[m_idx][sort_by_, 2], c=colors[m_idx][sort_by_], marker='o', edgecolors='k', s=80)
ax3.set_xlabel('PC2', fontsize=24)
ax3.set_ylabel('PC3', fontsize=24)
ax3.tick_params(axis='both', which='major', labelsize=20)
ax3.set_ylim([-0.1, 0.2])
ax3.set_yticks(np.arange(-0.1, 0.21, 0.1))

tick_range = np.arange(0.0, 1.0+0.01, 0.1)
cbar = plt.colorbar(sct, ax=cbar_ax, ticks=tick_range, location='right', fraction=2.5)
cbar.set_label(r'Mach number, M', fontsize=24)
cbar.set_ticklabels(np.round(tick_range*(m_sweep_range[-1]-m_sweep_range[0])+m_sweep_range[0], 2))
cbar.ax.tick_params(axis='both', which='major', labelsize=20)
sort_by_ = np.argsort(rX_pca[m_idx][:, 1])
sct = ax4.scatter(rX_pca[m_idx][sort_by_, 0], rX_pca[m_idx][sort_by_, 2], c=colors[m_idx][sort_by_], marker='o', edgecolors='k', s=80) 
ax4.set_xlabel('PC1', fontsize=24)
ax4.set_ylabel('PC3', fontsize=24)
ax4.tick_params(axis='both', which='major', labelsize=20)
ax4.set_ylim([-0.1, 0.2])
ax4.set_xticks(np.arange(-0.25, 0.25+.1, 0.25))
ax4.set_yticks(np.arange(-0.1, 0.21, 0.1))
sct.set_cmap('viridis') 
plt.tight_layout()
# plt.savefig('./Saved Figures/4_latvar_m.png', bbox_inches='tight', dpi=300)

### Aerodynamic Coefficients - A Monte Carlo Approach

Functions to calculate $C_n$, $C_a$, $C_m$

In [ ]:
import scipy
upper_xc_af = np.flip(np.array(data_raw.values[0,:28], dtype = 'float'))
lower_xc_af = np.array(data_raw.values[0, 28:28*2], dtype = 'float')

def brute_cn(samples, samples_af):
    cn = []
    u_bound = samples_af.shape[0]//2+1
    l_bound = samples_af.shape[0]//2
    samples = samples.detach().numpy()
    upper_xc = (samples_af[:u_bound,-2].cpu().detach().numpy()+1)/2
    lower_xc = np.flip(samples_af[l_bound:,-2].cpu().detach().numpy()+1)/2
    yu = np.interp(x = upper_xc, xp=upper_xc_af, fp=np.flip(samples_af[0,:28].cpu().detach().numpy()/y_scale))
    yl = np.interp(x = lower_xc, xp=lower_xc_af         , fp=samples_af[0, 28:28*2].cpu().detach().numpy()/y_scale)

    dyudx = np.gradient(yu, upper_xc)
    dyldx = np.gradient(yl, lower_xc)
    
    for i in np.arange(0, samples.shape[0]):
        cn.append(-np.trapz(samples[i, :u_bound]/y_scale, x=upper_xc) + np.trapz(np.flip(samples[i, l_bound:])/y_scale, x=lower_xc) ) 
    return np.mean(cn), np.std(cn)

def brute_cm(samples, samples_af, loc=0.25):
    cl = []
    samples = samples.detach().numpy()
    u_bound = samples_af.shape[0]//2+1
    l_bound = samples_af.shape[0]//2
    upper_xc = (samples_af[:u_bound,-2].cpu().detach().numpy()+1)/2
    lower_xc = np.flip(samples_af[l_bound:,-2].cpu().detach().numpy()+1)/2

    yucs = scipy.interpolate.CubicSpline(upper_xc_af, np.flip(samples_af[0,:28].cpu().detach().numpy()/y_scale))
    yu = yucs(upper_xc)
    ylcs = scipy.interpolate.CubicSpline(lower_xc_af, samples_af[0, 28:28*2].cpu().detach().numpy()/y_scale)
    yl = ylcs(lower_xc)

    dyudx = np.gradient(yu, upper_xc, edge_order=1)
    dyldx = np.gradient(yl, lower_xc, edge_order=1)
    
    # print(dyudx)    
    for i in np.arange(0, samples.shape[0]):
        term1 = -np.trapz(samples[i, :u_bound]/y_scale*(loc - upper_xc), x=upper_xc) + np.trapz(np.flip(samples[i, l_bound:])/y_scale*(loc - lower_xc), x=lower_xc)
        term2 = np.trapz(samples[i, :u_bound]/y_scale * dyudx * yu, x=upper_xc) - np.trapz(samples[i, l_bound:]/y_scale * dyldx * yl, x=lower_xc)
        cl.append(term1 + term2)
    return np.mean(cl), np.std(cl)

def brute_ca(samples, samples_af, manual_coords = None):
    ca = []
    samples = samples.detach().numpy()
    u_bound = samples_af.shape[0]//2+1
    l_bound = samples_af.shape[0]//2
    upper_xc = (samples_af[:u_bound,-2].cpu().detach().numpy()+1)/2
    lower_xc = np.flip(samples_af[l_bound:,-2].cpu().detach().numpy()+1)/2
    yucs = scipy.interpolate.CubicSpline(upper_xc_af, np.flip(samples_af[0,:28].cpu().detach().numpy()/y_scale))
    yu = yucs(upper_xc)
    ylcs = scipy.interpolate.CubicSpline(lower_xc_af, samples_af[0, 28:28*2].cpu().detach().numpy()/y_scale)
    yl = ylcs(lower_xc)
    if manual_coords is None:
        dyudx = np.gradient(yu, upper_xc, edge_order=2)
        dyldx = np.gradient(yl, lower_xc, edge_order=2)
    else:
        dyudx = np.interp(upper_xc, manual_coords[0], np.gradient(manual_coords[1], manual_coords[0], edge_order=1))
        dyldx = np.interp(lower_xc, manual_coords[2], np.gradient(manual_coords[3], manual_coords[2], edge_order=1))
    for i in np.arange(0, samples.shape[0]):
        ca.append(np.trapz(samples[i, :u_bound]/y_scale*dyudx, x=upper_xc) - np.trapz(np.flip(samples[i, l_bound:]/y_scale)*dyldx, x=lower_xc)) 
    return np.mean(ca), np.std(ca)

def brute_cd(samples, samples_af, ang):
    cd = []
    samples = samples.detach().numpy()
    u_bound = samples_af.shape[0]//2+1
    l_bound = samples_af.shape[0]//2
    upper_xc = (samples_af[:u_bound,-2].cpu().detach().numpy()+1)/2
    lower_xc = np.flip(samples_af[l_bound:,-2].cpu().detach().numpy()+1)/2
    yucs = scipy.interpolate.CubicSpline(upper_xc_af, np.flip(samples_af[0,:28].cpu().detach().numpy()/y_scale))
    yu = yucs(upper_xc)
    ylcs = scipy.interpolate.CubicSpline(lower_xc_af, samples_af[0, 28:28*2].cpu().detach().numpy()/y_scale)
    yl = ylcs(lower_xc)#np.interp(x = lower_xc,
    dyudx = np.gradient(yu, upper_xc, edge_order=2)
    dyldx = np.gradient(yl, lower_xc, edge_order=2)
    
    for i in np.arange(0, samples.shape[0]):
        u = np.trapz(y = samples[i, :u_bound]/y_scale*(dyudx - np.tan(ang)), x=upper_xc).flatten()
        l = np.trapz(y = samples[i, l_bound:]/y_scale*(dyldx - np.tan(ang)), x=lower_xc).flatten()
        cd.append(u - l)
    return np.mean(cd), np.std(cd)

#### Calculate Coefficients

In [ ]:
def calculate_coeffs(airfoil, a, m, num_samples):
    from itertools import product 
    
    # Check a and m format
    if not (isinstance(a, list) or isinstance(a, np.ndarray)):
        a = [a]
    if not (isinstance(m, list) or isinstance(m, np.ndarray)):
        m = [m]
    iter_list = list(product(m, a))
    # Set up 
    CL, CL_2std = [], [] 
    CD, CD_2std = [], []
    CM, CM_2std = [], []

    for t in tn.tqdm(range(len(iter_list))):
        ang = iter_list[t][1]
        mach = iter_list[t][0]
        # Set up samples 
        sample_airfoil = gen_test_data(airfoil, np.deg2rad(ang), mach, manual_point=None)
        pred_debug = aggregate_posterior(sample_airfoil.to(output_device), weights_)
        pred_debug_samples = pred_debug.sample(sample_shape=torch.Size([num_samples]))
        
        # Calculate Cn, Ca, Cm
        Cn_mu, Cn_std = brute_cn(pred_debug_samples, sample_airfoil) 
        Ca_mu, Ca_std = brute_ca(pred_debug_samples, sample_airfoil)
        Cm_mu, Cm_std = brute_cm(pred_debug_samples, sample_airfoil)
        
        # Get Cl and std
        CL.append(Cn_mu*np.cos(np.deg2rad(ang)) - Ca_mu*np.sin(np.deg2rad(ang)))
        # - np.sin(np.deg2rad(ang))*np.tan(np.deg2rad(ang)) this for Eq. from Flemming 1984 (more acc)
        CL_2std.append(2*np.sqrt((Cn_std*np.abs(np.cos(np.deg2rad(ang))))**2 + (Ca_std*np.abs(np.sin(np.deg2rad(ang))))**2))
        # Get Cd and std
        CD.append(Cn_mu*np.sin(np.deg2rad(ang)) + Ca_mu*(np.cos(np.deg2rad(ang))))
        # - np.sin(np.deg2rad(ang))*np.tan(np.deg2rad(ang))
        CD_2std.append(2*np.sqrt((Cn_std*np.abs(np.sin(np.deg2rad(ang))))**2 + (Ca_std*np.abs(np.cos(np.deg2rad(ang))))**2))
        # Get Cm and std
        CM.append(Cm_mu)
        CM_2std.append(2*Cm_std)

    return (np.array(CL), np.array(CL_2std)), (np.array(CD), np.array(CD_2std)), (np.array(CM), np.array(CM_2std)), np.array(iter_list)

#### Example: SC1095

Calculate

In [ ]:
model = model.cuda()
a_range = np.arange(-4.0, 13.0)
# Calculate airfoil info 
a, b, c, d = calculate_coeffs('SC 1095', a_range, [0.40, 0.60, 0.70], 5000)

# M = 0.40 
pred_Cl_list_m040, pred_Cl_2std_list_m040 = a[0][d[:,0]==0.4], a[1][d[:,0]==0.4]
pred_Cd_list_m040, pred_Cd_2std_list_m040 = b[0][d[:,0]==0.4], b[1][d[:,0]==0.4]
pred_Cm_list_m040, pred_Cm_2std_list_m040 = c[0][d[:,0]==0.4], c[1][d[:,0]==0.4]

# M = 0.60 
pred_Cl_list_m060, pred_Cl_2std_list_m060 = a[0][d[:,0]==0.6], a[1][d[:,0]==0.6]
pred_Cd_list_m060, pred_Cd_2std_list_m060 = b[0][d[:,0]==0.6], b[1][d[:,0]==0.6]
pred_Cm_list_m060, pred_Cm_2std_list_m060 = c[0][d[:,0]==0.6], c[1][d[:,0]==0.6]


Expt Data for validation (Brute forcing)


In [ ]:
a_sc1095_m060 = np.array([-0.19, -5.30, -3.37, -1.41, -0.30, 3.14, 6.22, 9.17, 10.27, 11.23, 12.24, 13.15, 14.20, 16.15, -0.11, -0.95])
a2_sc1095_m060 = np.array([0.03, 3.16, 6.28, 9.31, 11.33, 9.31, 6.28, 3.16, 0.03, -3.11, 0.02]) # run 33
cd2_sc1095_m060 = np.array([np.nan, np.nan, 0.0084, 0.0718, 0.1104, 0.0708, 0.0098, np.nan, np.nan, np.nan, np.nan])
cl2_sc1095_m060 = np.array([0.12, 0.485, 0.814, 0.937, 0.979, 0.935, 0.811, 0.483, 0.116, -0.286, 0.099])
cm2_sc1095_m060 = np.array([-0.019, -0.016, -0.001, -0.012, -0.31, -0.011, 0, -0.015, -0.019, -0.022, -0.019])
cl_sc1095_m060 = np.array([0.092, -0.505, -0.300, -0.049, 0.078, 0.489, 0.826, 0.930, 0.966, 0.979, 0.947, 0.951, 0.959, 0.956, 0.09, -0.009])
cd_sc1095_m060 = np.array([0.0074, 0.0114, 0.0073, 0.0075, 0.0072, 0.0084, 0.0218, 0.0636, 0.0587, 0.0715, 0.1273, 0.1501, 0.1633, 0.1675, 0.0075, -0.009])
cd_sc1095_m060 = np.array([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 0.0316, 0.0822, 0.0931, 0.1129, 0.1322, 0.1491, 0.1713, 0.1976, np.nan, np.nan])
cm_sc1095_m060 = np.array([-0.02, -0.029, -0.024, -0.020, -0.020, -0.016, -0.001, -0.013, -0.023, -0.034, -0.048, -0.059, -0.071, -0.092, -0.019, -0.021]) 

a_sc1095_m040 = np.array([0.05, -5.05, -3.16, -1.24, -0.17, 2.98, 6.13, 9.09, 11.1, 12.06, 12.97]) #  , 13.94, 14.89])
cl_sc1095_m040 = np.array([0.110, -.424, -.231, -0.027, 0.092, 0.418, 0.745, 1.041, 1.211, 1.243, 0.874]) #, 0.852, 0.852])
cl_sc1095_m040 = np.array([0.132, -0.432, -0.194, -0.003, 0.098, 0.414, 0.759, 1.056, 1.239, 1.293, 0.695]) # Balance
cd_sc1095_m040 = np.array([0.0076, 0.0079, 0.0071, 0.0075, 0.0069, 0.008, 0.008, 0.0108, 0.0138, 0.0165, 0.2007])#, 0.2213, 0.228, 0.1042, 0.1264, 0.2253]) 
cm_sc1095_m040 = np.array([-0.017, -0.02, -0.019, -0.017, -0.017, -0.015, -0.012, -0.005, 0.003, 0.012, -0.107])#, -0.108, -0.0110])
a2_sc1095_m040 = np.array([-0.27, -5.01, -3.38, -1.36, -0.32, 2.98, 5.96, 9.03, 11.05, 12.05, 12.85, 13.94]) # Run 16
cm2_sc1095_m040 = np.array([-0.01, -0.035, -0.030, -0.022, -0.018, -0.008, -0.002, 0.007, 0.015, 0.01, -0.088, -0.091])
cd2_sc1095_m040 = np.array([-0.01, -0.035, -0.030, -0.022, -0.018, -0.008, -0.002, 0.007, 0.015, 0.01, -0.088, -0.091])

a_sc1095_m070 = np.array([0.06, -5.17, -3.35, -1.20, -0.17, 2.15, 3.26, 6.31, 8.31, 9.27, 10.26, 11.28])
cl_sc1095_m070 = np.array([0.128, -0.595, -.341, -0.063, 0.089, 0.397, 0.55, 0.802, 0.850, 0.883, 0.9, 0.901]) # np.array([0.117, -0.614, -0.347, -0.054, 0.099, 0.421, 0.579, 0.840, 0.852, 0.873, 0.876, 0.870])#
cl_sc1095_m070 = np.array([0.117, -0.614, -0.347, -0.054, 0.099, 0.421, 0.579, 0.840, 0.852, 0.873, 0.876, 0.870]) # Balance
cd_sc1095_m070 = np.array([0.0074, 0.0335, 0.0143, 0.0077, 0.0074, 0.0087, 0.0144, 0.0529, 0.0699, 0.0729, 0.0778, 0.089])
cd_sc1095_m070 = np.array([0.0068, 0.0461, 0.330, 0.0136, 0.0095, np.nan, 0.0145, 0.0513, 0.0785, 0.0955, 0.1094, 0.1201]) # Balance
cm_sc1095_m070 = np.array([-0.021, -0.027, -0.029, -0.022, -0.021, -0.014, -0.018, -0.041, -0.051, -0.047, -0.059, -0.067])
a2_sc1095_m070 = np.array([-0.32, -5.25, -3.53, -1.34, -0.21, 2.3])
cm2_sc1095_m070 = np.array([-0.012, -0.035, -0.024, -0.012, -0.007, 0.004])
cd2_sc1095_m070 = np.array([0.0071, 0.0323, 0.0135, .0077, 0.0069, 0.0083])

Plot

In [ ]:
a_range = np.arange(-4.0, 13.0)
plt.figure(figsize=(8,6))
plt.errorbar(a_sc1095_m040, cl_sc1095_m040, yerr=np.ones_like(cl_sc1095_m040)*0.022*2, fmt='o', color='blue', linewidth=2.0, capsize=2.0, label='M = 0.40, Expt.')
plt.errorbar(a_sc1095_m060, cl_sc1095_m060, yerr=np.ones_like(cl_sc1095_m060)*0.007*2, fmt='s', color='orangered', linewidth=2.0, capsize=2.0, label='M = 0.60, Expt.')
plt.errorbar(a2_sc1095_m060, cl2_sc1095_m060, yerr=np.ones_like(cl2_sc1095_m060)*0.007*2, fmt='s', color='orangered', linewidth=2.0, capsize=2.0)

plt.plot(a_range, pred_Cl_list_m040, label='M = 0.40, LAM', linewidth=2.0)
plt.plot(a_range, pred_Cl_list_m060, label='M = 0.60, LAM', linewidth=2.0)
plt.fill_between(a_range, pred_Cl_list_m040 - pred_Cl_2std_list_m040, pred_Cl_list_m040 + pred_Cl_2std_list_m040, alpha=0.2)
plt.fill_between(a_range, pred_Cl_list_m060 - pred_Cl_2std_list_m060, pred_Cl_list_m060 + pred_Cl_2std_list_m060, alpha=0.2)
plt.legend()
plt.xlabel(r'$\alpha$ [deg]', fontsize=36)
plt.ylabel('$c_l$', fontsize=36)
plt.xticks(fontsize=32)
plt.yticks(np.arange(-.75, 1.3, 0.5), fontsize=32)
plt.xlim([-4.0, 12.0])
plt.ylim([-.75, 1.3])
plt.legend(loc='lower right', ncols = 2, columnspacing=0.5, handletextpad=0.2, handlelength=1, fontsize=22)
plt.tight_layout()
# plt.savefig("./Saved Figures/4_cl_vs_alpha_sc1095.pdf", dpi=300, )

plt.figure(figsize=(8,6))
plt.errorbar(a_sc1095_m040, cd_sc1095_m040, yerr=np.ones_like(cl_sc1095_m040)*0.017*2, fmt='o', color='blue', linewidth=2.0, capsize=2.0, label='M = 0.40, Expt.')
plt.errorbar(a_sc1095_m060, cd_sc1095_m060, yerr=np.ones_like(cl_sc1095_m060)*0.006*2, fmt='s', color='orangered', linewidth=2.0, capsize=2.0, label='M = 0.60, Expt.')
plt.errorbar(a2_sc1095_m060, cd2_sc1095_m060, yerr=np.ones_like(cd2_sc1095_m060)*0.006*2, fmt='s', color='orangered', linewidth=2.0, capsize=2.0)
plt.plot(a_range, pred_Cd_list_m040, label='M = 0.40, LAM', linewidth=2.0)
plt.plot(a_range, pred_Cd_list_m060, label='M = 0.60, LAM', linewidth=2.0)
plt.fill_between(a_range, pred_Cd_list_m040 - pred_Cd_2std_list_m040, pred_Cd_list_m040 + pred_Cd_2std_list_m040, alpha=0.2)
plt.fill_between(a_range, pred_Cd_list_m060 - pred_Cd_2std_list_m060, pred_Cd_list_m060 + pred_Cd_2std_list_m060, alpha=0.2)
plt.legend()
plt.xlabel(r'$\alpha$ [deg]', fontsize=36)
plt.ylabel('$c_d$', fontsize=36)
plt.legend(loc='upper left', ncols = 2, columnspacing=0.5, handletextpad=0.2, handlelength=1, fontsize=22)
plt.xlim([-4.0, 12.0])
plt.ylim([-0.05, 0.3])
plt.xticks(fontsize=32)
plt.yticks(np.arange(-.05, 0.31, 0.05), fontsize=32)
plt.tight_layout()
# plt.savefig("./Saved Figures/4_cd_vs_alpha_sc1095.pdf", dpi=300,)

plt.figure(figsize=(8,6))
plt.errorbar(a_sc1095_m040, cm_sc1095_m040, yerr=np.ones_like(cm_sc1095_m040)*0.009*2, fmt='o', color='blue', linewidth=2.0, capsize=2.0, label='M = 0.40, Expt.')
plt.errorbar(a2_sc1095_m040, cm2_sc1095_m040, yerr=np.ones_like(cm2_sc1095_m040)*0.009*2, fmt='o', color='blue', linewidth=2.0, capsize=2.0)
plt.errorbar(a_sc1095_m060, cm_sc1095_m060, yerr=np.ones_like(cm_sc1095_m060)*0.003*2, fmt='s', color='orangered', linewidth=2.0, capsize=2.0, label='M = 0.60, Expt.')
plt.errorbar(a2_sc1095_m060, cm2_sc1095_m060, yerr=np.ones_like(cm2_sc1095_m060)*0.003*2, fmt='s', color='orangered', linewidth=2.0, capsize=2.0)
plt.plot(a_range, pred_Cm_list_m040, label='M = 0.40, LAM', linewidth=2.0)
plt.plot(a_range, pred_Cm_list_m060, label='M = 0.60, LAM', linewidth=2.0)
plt.fill_between(a_range, pred_Cm_list_m040 - pred_Cm_2std_list_m040, pred_Cm_list_m040 + pred_Cm_2std_list_m040, alpha=0.2)
plt.fill_between(a_range, pred_Cm_list_m060 - pred_Cm_2std_list_m060, pred_Cm_list_m060 + pred_Cm_2std_list_m060, alpha=0.2)
plt.ylim([-0.15, 0.05])
plt.legend(loc='lower left', ncols = 2, columnspacing=0.5, handletextpad=0.2, handlelength=1, fontsize=22)
plt.xlabel(r'$\alpha$ [deg]', fontsize=36)
plt.ylabel('$c_m$', fontsize=36)
plt.xlim([-4.0, 12.0])
plt.xticks(fontsize=32)
plt.yticks(np.arange(-.15, 0.05+.01, 0.05), fontsize=32)
plt.tight_layout()
# plt.savefig("./Saved Figures/4_cm_vs_alpha_sc1095.pdf", dpi=300,)